In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)

In [12]:
# Gets the Drug Label
query = """
SELECT ?label
WHERE {
  wd:Q220426 rdfs:label ?label.
  FILTER (lang(?label) = "en")
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['label']['value'])

zopiclone


In [76]:
# Get All Interacting Drugs wrt. another Drug (e.g., Zopiclone)
drug = "wd:Q220426"
interaction_property = "wdt:P769"
side_effect_property = "wdt:P1909"

query = f"""
SELECT ?interactingDrugLabel 
WHERE {{
  {drug} {interaction_property} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for interacting_drug in results['results']['bindings']:
    print(interacting_drug['interactingDrugLabel']['value'])

codeine
pethidine
loxapine
morphine
alfentanil
(RS)-methadone
hydromorphone
dihydrocodeine
oxycodone
fentanyl
sodium oxybate
buprenorphine
tapentadol
sufentanil
periciazine
tramadol


##### p:P769 is used to find the statement node describing a significant drug interaction.
##### ps:P769 extracts the drug involved in the interaction (e.g., Pethidine, Periciazine).
##### pq:P1909 extracts the side effects associated with the interaction (e.g., "Depressed level of consciousness").

In [74]:
# Get Side Effects of a Drug (e.g., Zopiclone)
drug = "wd:Q220426"
interaction_property = "p:P769"
interaction_statement = "ps:P769"
side_effect_qualifier = "pq:P1909"

query = f"""
SELECT ?interactingDrugLabel (GROUP_CONCAT(?sideEffectLabel; separator=", ") AS ?sideEffects)
WHERE {{
  {drug} {interaction_property} ?statement.
  ?statement {interaction_statement} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")
  
  OPTIONAL {{
    ?statement {side_effect_qualifier} ?sideEffect.
    ?sideEffect rdfs:label ?sideEffectLabel.
    FILTER (LANG(?sideEffectLabel) = "en")
  }}
}}
GROUP BY ?interactingDrugLabel
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

i = 1
for binding in results['results']['bindings']:
    interacting_drug_label = binding['interactingDrugLabel']['value']
    side_effects = binding.get('sideEffects', {}).get('value', 'No side effects listed')
    print(f"{(str(i) + '.').ljust(3)} Chemical Substance: {interacting_drug_label.upper().ljust(15)} | Side Effects: {side_effects}")
    i += 1

1.  Chemical Substance: PETHIDINE       | Side Effects: depressed level of consciousness, central respiratory depression
2.  Chemical Substance: PERICIAZINE     | Side Effects: depressed level of consciousness
3.  Chemical Substance: TRAMADOL        | Side Effects: depressed level of consciousness, central respiratory depression
4.  Chemical Substance: (RS)-METHADONE  | Side Effects: depressed level of consciousness, central respiratory depression
5.  Chemical Substance: MORPHINE        | Side Effects: depressed level of consciousness, central respiratory depression
6.  Chemical Substance: OXYCODONE       | Side Effects: depressed level of consciousness, central respiratory depression
7.  Chemical Substance: HYDROMORPHONE   | Side Effects: depressed level of consciousness, central respiratory depression
8.  Chemical Substance: DIHYDROCODEINE  | Side Effects: depressed level of consciousness, central respiratory depression
9.  Chemical Substance: CODEINE         | Side Effects: depresse

In [ ]:
# Get all instances of "medicines"
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i=1
meds = []
for drug in results['results']['bindings']:
    print(str(i) + ":", drug['drugLabel']['value'])
    meds.append(drug['drugLabel']['value'])
    i += 1

1: gabapentin
2: rac-salbutamol
3: isosorbide dinitrate
4: ampicillin
5: Diclofenac
6: D-sorbitol
7: isoflurophate
8: 6-thioguanine
9: desoximetasone
10: ajmaline
11: natalizumab
12: ethanol
13: L-menthol
14: aluminum hydroxide
15: chorionic gonadotropin
16: lercanidipine
17: megestrol
18: neostigmine
19: boceprevir
20: benzylpenicillin
21: cilostazol
22: fusidic acid
23: DL-amphetamine
24: sodium bicarbonate
25: phenobarbital
26: theophylline
27: mazindol
28: aminocaproic acid
29: alpha-linolenic acid
30: nimesulide
31: biotin
32: L-glutamine
33: magnesium hydroxide
34: tramadol
35: iniparib
36: tioconazole
37: ibudilast
38: isoconazole
39: aprindine
40: nevirapine
41: ciprofloxacin
42: α-streptozocin
43: doxorubicin
44: azathioprine
45: carbon dioxide
46: amifostine
47: cyanocobalamin
48: lansoprazole
49: L-serine
50: carbamazepin
51: modafinil
52: arsenic trioxide
53: adenine
54: metformin
55: root vegetable
56: furosemide
57: eprazinone
58: obinutuzumab
59: agomelatine
60: aspirin


In [47]:
# Get the count of all medicines
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT (COUNT(?drug) AS ?drugCount)
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print("Medicine Count:", results['results']['bindings'][0]['drugCount']['value'])


Medicine Count: 2689


In [55]:
# Find Chemical Substance used as Medicine through Query by name
query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug rdfs:label "zopiclone"@en.
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['drug']['value'])

http://www.wikidata.org/entity/Q220426


In [ ]:
# Get Best Matched Meds based on Wrongly Type Input Medication
from fuzzywuzzy import process

input_name = "zopliklon"  

matches = process.extract(input_name, meds, limit=5)
print(matches)


[('zopiclone', 78), ('eszopiclone', 70), ('xylometazoline', 60), ('tetrahydrozoline', 60), ('zotepine', 59)]
